# Listas ligadas ó Linked lists

## Tabla de contenido
- [**Objetivo**](#Objetivo)
- **Introducción**

**Nota: la tabla de contenido se va a generar una vez que se tenga la versión final de este notebook, para no perder tiempo rehaciendola en cada cambio.**

## Objetivo
Entender la implementación de las listas ligadas 

## Introducción
Una lista es una estructura de datos similar a un arreglo, de hecho podría decirse, de una forma laxa, que las listas son arreglos de tamaño dinámico.
A grandes rasgos una lista ligada es una serie de objetos ligados, donde el elemento en la posición <em>n</em> tiene una referencia al objeto en la posición *n+1*. De este modo para acceder al a cualquier elemento se puede seguir la cadena de referencias desde el primer elemento hasta el _n-ésimo_ elemento que queramos accesar.

![Ejemplo lista](https://raw.githubusercontent.com/sainoba/py-notas-estructuras/master/Notebook/ListasLigadas/img/Lista.svg?sanitize=true)

Una lista ligada tiene las siguientes características:
- Sus elementos están ordenados secuencialmente.
- Se pueden insertar nuevos elementos en cualquier posición.
- Se puede acceder/remover cualquier elemento.
- Puede crecer/reducir su tamaño en tiempo de ejecución. Contrario a lo que pasa con un arreglo.
- En varios lenguajes de programación permite guardar objetos de tipos distintos.

Pareciera que una lista ligada es una versión mejorada de los arreglos. No necesitamos declarar un tamaño al momento de crearla, e inlcuso podemos guardar objetos de distintos tipos.

## ¿Por qué no sustituimos _arreglos_ por _listas_ en todo?
Ignorando código trivialmente optimizable llega un punto en el que una estructura de datos no puede ganar ciertas características sin sacrificar otras. Se tienen que comprometer propiedades por otras, esto es conocido como [trade-off](#Trade-off).
Las listas sacrifican velocidad de accesso para ganar tamaño dinámico. Esto significa que suele ser más lento accesar elementos guardados en la lista, pero podremos agregar y remover elementos de esta sin preocuparnos de quedarnos sin espacio[*](#Espacio-ilimitado).

Esto pasa porque los elementos guardados en la lista muy probablemente no estén alojados en secciones contiguas de memoria, incluso pueden llegar a ocupar tamaños distintos en la memoria por lo que no se puede hacer uso de un polinomio de direccionamiento. Esto significa que tenemos que seguir la cadena de referencias, en vez de hacer un simple cálculo.

## Métodos
Una lista requiere de al menos 2 funciones:
- **insert(indice, elemento)**: Inserta un elemento en la posición solicitada.
- **remove(indice)**: Remueve y regresa un elemento en la posición solicitada.

## Métodos adicionales
Adicionalmente se suelen implementar las siguientes funciones por comodidad:
- **get(indice)**: Regresa el elemento en la posición solicitada.
- **append(elemento)**: Inserta un elemento al final de la lista.
- **length()**: Regresa la cantidad de elementos contenidos en la lista.

## Esqueleto
Tomando como referencia estas funciones podemos empezar a armar un esqueleto de nuestra estructura Lista.
Necesitaremos implementar los siguientes métodos
- **`__init__`**: Método para preparar nuestra lista antes de usarla. [Checar notas](#Función-<code>__init__<%2Fcode>)
- **`__len__`**: Método para regresar la cantidad de elementos en la lista. [Checar notas](#Función-<code>__len__<%2Fcode>)
- **`__getitem__`**: Método para obtener un elemento en la posición solicitada. [Checar notas](#Función-<code>__getitem__<%2Fcode>)
- **`insert`**, **`remove`**, **`append`**: Métodos descritos previamente.


In [1]:
class Lista:
    def __init__(self):  # Prepara la instancia de la clase para poder usarla
        pass

    def __len__(self):  # Regresa la longitud de la lista
        pass

    def __getitem__(self, indice):  # Regresa el objeto en la posición solicitada
        pass
    
    def insert(self, indice, objeto):  # Inserta un elemento en la posición solicitada
        pass
    
    def remove(self, indice):  # Remueve y regresa el elemento en la posición solicitada
        pass

    def append(self, objeto):  # Inserta un elemento al final de la lista
        pass

En el código anterior defininimos nuestra clase Lista con métodos vacios, casi listos para ser implementados. El primer paso es describir como implementaremos las referencias dentro de nuestra lista.

## Implementación con objetos ligados
Esta implementación se basa en dos puntos:
1. Se guarda la referencia al primer objeto.
2. Cada objeto guarda una referencia al siguiente.

<img src="https://raw.githubusercontent.com/sainoba/py-notas-estructuras/master/Notebook/ListasLigadas/img/ObjetosLigados.svg?sanitize=true">

Como se ve en la imagen, si tenemos una forma de llegar al primer elemento podemos llegar a cualquier otro siguiendo las referencias al siguiente. Por eso el nombre de **objetos ligados**.
Aquí nuestro _primer elemento_ sería el que se encuentra más a la izquierda.

Esta estrategia de objetos ligados se ve muy bien pero tiene un pequeño detalle, para guardar una referencia al _siguiente objeto_ necesitamos agregar una variable a cada objeto que se guarde. No es para nada recomendable modificar los objetos a guardar, afortunadamente podemos encapsular los objetos en una clase contenedora, que además de contener al objeto, también incluye la referencia al siguiente elemento de la lista:

<center><img src="https://raw.githubusercontent.com/sainoba/py-notas-estructuras/master/Notebook/ListasLigadas/img/ContenedorLigado.svg?sanitize=true"></center>

De esta manera podemos conservar nuestros objetos originales intactos, y también podemos guardar la referencia al siguiente.

Vamos a programar nuestra estructura paso a paso.
### Paso 1: Definir un contenedor
Aquí vamos a definir nuestra clase contenedora, esta clase es muy sencilla ya que sólo vamos a definir 2 variables que guarden las 2 cosas que nos interesan:
- Una referencia al siguiente contenedor.
- El objeto que queremos guardar.

In [ ]:
class Contenedor:
    def __init__(self, objeto, siguiente=None):  # Se prepara al objeto contenedor para que pueda ser usado
        self.siguiente = siguiente  # Se guarda la referencia al siguiente contenedor
        self.objeto = objeto  # Se guarda al objeto que se quiere encapsular

Como podemos observar nuestro _objeto_ a guardar no se modifica.
### Paso 2: Implementar método `__init__` de la Lista
Este método sirve para preparar nuestra estructura antes de que sea usada. En este caso en particular vamos a guardar 3 cosas:
- Un contador que indica el número de elementos en la lista, este aumenta o disminuye cada vez que hacemos `insert(...)` o `remove(...)`
- La variable _primero_, que es una referencia al primer elemento de la lista.
- La variable _ultimo_, que es una referencia al último elemento de la lista.

In [3]:
class Pila:
    def __init__(self):  # Preparamos nuestra estructura
        self.cuantos = 0  # La lista tiene 0 elementos cuando se crea
        self.primero = None  # La lista no tiene primero elemento, porque está vacía
        self.ultimo = None  # La lista no tiene último elemento, porque está vacía

¿Por qué usamos la variable `cuantos` si podemos recorrer la lista y contar los elementos?</br>
No ocupa mucho espacio y es más eficiente que recorrer toda la lista para contar los elementos cada vez que queremos saber su longitud.

¿Por qué usamos la variable `ultimo`?</br>
Esta variable nos facilitará la implementación del método `append`.



### Paso 3: Implementar método `__len__`
Este método regresa la cantidad de elementos en nuestra lista. Afortunadamente ya tenemos una variable que se encarga de llevar la cuenta. Por eso mismo sólo es cuestión de regresarla.
Sólo tenemos que tener cuidado de actualizar `cuantos` cada vez que se haga un `insert` o un `remove`.

In [ ]:
class Pila:
    def __init__(self):  # Preparamos nuestra estructura
        self.cuantos = 0  # La lista tiene 0 elementos cuando se crea
        self.primero = None  # La lista no tiene primero elemento, porque está vacía
        self.ultimo = None  # La lista no tiene último elemento, porque está vacía

    def __len__(self):  # Método para regresar la longitud de la lista
        return self.cuantos  # Regresamos nuestro contador

### Paso 4: Implementar método `__getitem__`
El método `__getitem__` nos regresará el objeto en cierta posición solicitada.

#### Caso 1 la lista está vacía:
Podemos hacer varias cosas como lanzar un error, o regresar un valor nulo(`None`). Ambos tienen ventajas y desventajas, en este caso decidimos lanzar un error.

#### Caso 2 la pila no está vacía:
Observemos que en nuestro método `__init__` creamos la variable `tope` que precisamente es una referencia al tope de la pila. Por lo que sólo basta con regresar este mismo.
Pero hay que tener cuidado, recordemos que nuestros objetos están dentro de contenedores, por esto mismo tenemos que sacarlos del contenedor usando `tope.objeto`.

#### Caso 3 la lista 

In [ ]:
class Pila:
    def __init__(self):  # Preparamos nuestra estructura
        self.cuantos = 0  # La lista tiene 0 elementos cuando se crea
        self.primero = None  # La lista no tiene primero elemento, porque está vacía
        self.ultimo = None  # La lista no tiene último elemento, porque está vacía

    def __len__(self):  # Método para regresar la longitud de la lista
        return self.cuantos  # Regresamos nuestro contador

    def insert(self, indice, objeto):  # Inserta un elemento en la posición solicitada
        if indice < 0 or indice > len(self):  # Verificamos que el índice sea valido
            raise IndexError('Índice inválido')  # Lanzamos un error

        elemento_anterior = None
        elemento_siguiente = self.primero  # Referencia que ayudará a recorrer la lista

        for idx in range(indice):
            elemento_anterior = elemento_siguiente
            elemento_siguiente = elemento_siguiente.siguiente

        elemento_nuevo = Contenedor(objeto, elemento_siguiente)

        if elemento_anterior is None:  # 
            self.primero = elemento_nuevo
            if self.ultimo is None:  # Al insertar el primer elemento este también se convierte en el último
                self.ultimo = self.primero
        else:
            elemento_anterior.siguiente = elemento_nuevo
        self.cuantos += 1
    
    
    def append(self, objeto):
        elemento_nuevo = Contenedor(objeto)
        if self.primero is None:
            self.primero = elemento_nuevo
        else:
            self.ultimo.siguiente = elemento_nuevo
        self.ultimo = elemento_nuevo
        self.cuantos += 1

    def remove(self, indice):  # Remueve y regresa el elemento en la posición solicitada
        if indice < 0 or indice >= len(self):  # Verificamos que el índice sea valido
            raise IndexError('Índice inválido')  # Lanzamos un error

        elemento_anterior = None
        elemento_actual = self.primero  # Referencia que ayudará a recorrer la lista

        for idx in range(indice):
            elemento_anterior = elemento_actual
            elemento_actual = elemento_actual.siguiente
            
        elemento_actual = None
        
        self.cuantos -= 1
        return elemento_actual.objeto
        
            

    def __getitem__(self, indice):  # Método para regresar el elemento en la posición solicitada
        if indice < 0 or indice > len(self):  # Verificamos que el índice sea válido
            raise IndexError('Índice inválido')  # Lanzamos un error
        elemento_tmp = self.primero  # Referencia que ayudará a recorrer la lista
        for idx in range(indice):  # Recorremos nuestra lista hasta llegar al índice deseado
            elemento_tmp = elemento_tmp.siguiente  # Se cmabia la referencia al siguiente en la lista
        return elemento_tmp.objeto  # Regresamos el objeto que se encuentra en el contenedor


### Paso 5: Implementar método `insert`
El método `insert` agrega un nuevo objeto a nuestra lista en cierta posición. Recordemos que para poder agragar un objeto tenemos que hacer 3 cosas:
1. Obtener una referencia al elemento _indice + 1_
1. Encapsular al objeto a insertar en un contenedor, incluyendo la referencia del punto 1
1. Cambiar la referencia de _indice - 1_ al nuevo elemento

hola
#### Caso 1 la lista está vacía:
Podemos hacer varias cosas como lanzar un error, o regresar un valor nulo(`None`). Ambos tienen ventajas y desventajas, en este caso decidimos lanzar un error.

#### Caso 2 la pila no está vacía:
Observemos que en nuestro método `__init__` creamos la variable `tope` que precisamente es una referencia al tope de la pila. Por lo que sólo basta con regresar este mismo.
Pero hay que tener cuidado, recordemos que nuestros objetos están dentro de contenedores, por esto mismo tenemos que sacarlos del contenedor usando `tope.objeto`.

#### Caso 3 la lista 

In [ ]:
class Pila:
    def __init__(self):  # Preparamos nuestra estructura
        self.cuantos = 0  # La lista tiene 0 elementos cuando se crea
        self.primero = None  # La lista no tiene primero elemento, porque está vacía
        self.ultimo = None  # La lista no tiene último elemento, porque está vacía

    def __len__(self):  # Método para regresar la longitud de la lista
        return self.cuantos  # Regresamos nuestro contador



### Implementaciones alternas @pendiente (Mencioral listas ligadas con nodos de más de un elemento)
Hay más de una forma de implementar pilas, cada una con ventajas y desventajas distintas.

## Notas de Python
### Función `__init__`
La función `__init__` se usa para preparar un objeto antes de empezar a usarlo.

En el caso de nuestra lista lo usaremos para crear e inicializar las variables que guardan información importante sobre la instancia, como el número de elementos dentro de la estructura.
### Función `__len__`
En Python se obtiene la longitud de un objeto con el siguiente código `len(objeto)`, esto Python lo termina convirtiendo a `objeto.__len__`.

Por eso mismo si quieremos que a nuestro objeto se le pueda preguntar su longitud implementamos `__len__`.

En este caso la longituda de nuestra Lista es la cantidad de elementos que tiene guardados.
### Función `__getitem__`
Python nos ofrece este método para accceder elementos indexados. Al sobreescribirlo podremos acceder a los objetos de una instancia de la siguiente manera `objeto[llave]`. Que es una alternativa más limpia a implementar `objeto.get(llave)`.

En nuestro caso lo usaremos para acceder a los elementos de una lista en cierta posición, por ejemplo: `mi_lista[13]`.

### Objeto `None`
Este objeto representa al objeto vacío. Es el análogo a `void` en C, o `null` en Java.

### Data model
Puedes encontrar más información sobre los métodos especiales de Python en lo documentación oficial: [Data model](https://docs.python.org/3/reference/datamodel.html).

## Notas generales
### Espacio ilimitado
No existe tal cosa como memoria infinita, cuando se refiere a que una estrucura puede crecer tanto como quiera significa que puede crecer tanto como el sistema lo permita. Puede estar limitada por la máquina virtual donde se ejecuta, como es el caso de Java, o en última instancia por el hardware.

## Definiciones
### Trade-off
Compromiso que se acuerda 

## Referencias
- Bader, D. (2017, June 27). Linked Lists in Python – dbader.org. Recuperado de https://dbader.org/blog/python-linked-list

@Pendiente (Darle formato a estas referenicas. vvvvvvvvvvv)
- Mehlhorn, K y Sanders P. (2008). Algorithms and Data Structures: The Basic Toolbox (pp. 8-12). doi: 10.1007/978-3-540-77978-0
- Mehlhorn, K. y Sanders, P. (2008). Algorithms and data structures: The basic toolbox. doi:10.1007/978-3-540-77978-0